This file explores the integration between listenbrainz, musicbrainz, and spotify.

It's separate from the application logic because the approach requires exploration to optimize and understand.
There's a lot more edge cases integratiing multiple edge cases than I had to contend with in using only Spotify.

Below, I sort out those edge cases, explore likelihood of error cases, so I can encode these states in tests.

Import Libs

In [ ]:
import os
import time
import json
import requests
from requests import HTTPError
from requests.auth import HTTPBasicAuth

Set up Script Variables

In [ ]:
artists = ["Noah Gundersen", "Julien Baker", "Phoebe Bridgers"]

Load Environment Variables

In [ ]:
listenbrainz_api_key = os.getenv("LISTENBRAINZ_API_KEY")
spotify_client_secret = os.getenv("SPOTIFY_CLIENT_SECRET")
spotify_client_id = os.getenv("SPOTIFY_CLIENT_ID")
# listenbrainz_api_key, spotify_client_secret, spotify_client_id

Provide useful context around request errors

In [ ]:
def handle_error(response):
    try:
        response.raise_for_status()
    except HTTPError as error:
        error.add_note(f"Request: {error.request}")
        error.add_note(f"Request headers: {error.request.headers}")
        error.add_note(f"Request body: {error.request.body}")
        error.add_note(f"Response: {error.response}")
        error.add_note(f"Response headers: {error.response.headers}")
        error.add_note(f"Response body: {error.response.text}")
        raise

Get Spotify Auth Token

In [ ]:
spotify_auth = requests.post(
    url="https://accounts.spotify.com/api/token",
    auth=HTTPBasicAuth(spotify_client_id, spotify_client_secret),
    data={"grant_type": "client_credentials"}
)
handle_error(spotify_auth)

auth_json = spotify_auth.json()
spotify_token = auth_json['access_token']
# spotify_token

Get playlist based on artists

In [ ]:
lb_radio = requests.get(
    url="https://api.listenbrainz.org/1/explore/lb-radio",
    params={
        "mode": "easy",
        "prompt": " ".join([f'artist:({artist})' for artist in artists])
    },
    headers={"Authorization": f"Bearer {listenbrainz_api_key}"}
)
handle_error(lb_radio)

tracks = lb_radio.json()["payload"]["jspf"]["playlist"]["track"]
len(tracks), tracks[:2]

In [ ]:
identifiers = []
for track in tracks:
    if len(track['identifier']) > 1:
        raise Exception(f'{track["title"]} by {track["creator"]} has multiple identifiers: {", ".join(track["identifiers"])}')
    identifiers += track["identifier"]
mbids = [x.split("/")[-1] for x in identifiers]
identifiers[:2], mbids[:2]

In [ ]:
all_relations = []
all_isrcs = []
spotify_tracks = []
rate_limit_remaining = 900  # Arbitrary > 0 number
rate_limit_max = 1000 # Slightly larger arbitrary > 0 number
for mbid in mbids:
    # time.sleep((rate_limit_max - rate_limit_remaining) / 1000)
    
    # https://musicbrainz.org/doc/MusicBrainz_API#Lookups
    mb_recording = requests.get(
        url=f"https://musicbrainz.org/ws/2/recording/{mbid}",
        params={"fmt": "json", "inc": "isrcs url-rels artists"},
        headers={"User-Agent": "mixtapestudy.com/0.0 ( douglas@builtonbits.com )"}
    )
    handle_error(mb_recording)
    
    recording_json = mb_recording.json()
    rate_limit_remaining = int(mb_recording.headers["X-RateLimit-Remaining"])
    rate_limit_max = int(mb_recording.headers["X-RateLimit-Limit"])
    print(f"{rate_limit_remaining}/{rate_limit_max}")
    
    relations = recording_json["relations"]
    isrcs = recording_json["isrcs"]
    artist_credit = recording_json["artist-credit"]

    if isrcs:
        query_string = f"isrc:{isrcs[0]}"
    elif relations:
        print(f'Finally found a relation! {[relation["url"]["resource"] for relation in relations]}')
              
    else:
        query_string = f'track:{recording_json["title"]}'
        if artist_credit:
            query_string += " " + " ".join([f'artist:{artist["artist"]["name"]}' for artist in artist_credit])
        
    print(f"query_string: {query_string}")
    
    # https://developer.spotify.com/documentation/web-api/reference/search
    spotify_search = requests.get(
        url="https://api.spotify.com/v1/search",
        params={"type": "track", "q": query_string},
        headers={"Authorization": f"Bearer {spotify_token}"}
    )
    handle_error(spotify_search)

    spotify_json = spotify_search.json()
    # print([t["id"] for t in spotify_json["tracks"]["items"]])
    if spotify_json["tracks"] and spotify_json["tracks"]["items"]:
        spotify_tracks.append(spotify_json["tracks"]["items"][0])
    
    all_relations += relations
    all_isrcs += isrcs
    
    # break

(len(all_relations), len(all_isrcs), len(spotify_tracks)), all_relations[:2], all_isrcs[:2]

Better song finding algorithm that avoids MusicBrainz rate limit

In [ ]:
spotify_tracks = []

for track in tracks:
    
    query_string = f'track:{track["title"]} artist:{track["creator"]}'
        
    # https://developer.spotify.com/documentation/web-api/reference/search
    spotify_search = requests.get(
        url="https://api.spotify.com/v1/search",
        params={"type": "track", "q": query_string},
        headers={"Authorization": f"Bearer {spotify_token}"}
    )
    handle_error(spotify_search)

    spotify_json = spotify_search.json()
    # print([t["id"] for t in spotify_json["tracks"]["items"]])
    if spotify_json["tracks"] and spotify_json["tracks"]["items"]:
        print("[X]", end=' ')
        spotify_tracks.append(spotify_json["tracks"]["items"][0])
        
    else:
        query_string = f'{track["title"]} {track["creator"]}'
        spotify_search = requests.get(
            url="https://api.spotify.com/v1/search",
            params={"type": "track", "q": query_string},
            headers={"Authorization": f"Bearer {spotify_token}"}
        )
        handle_error(spotify_search)
        
        spotify_json = spotify_search.json()
        if spotify_json["tracks"] and spotify_json["tracks"]["items"]:
            print("[/]", end=' ')
            spotify_tracks.append(spotify_json["tracks"]["items"][0])
        else:
            print("[ ]", end=' ')
        
    print(f"query_string: {query_string}", end=" | ")
    track = spotify_json["tracks"]["items"][0]
    print(f'{track["name"]} - {track["artists"][0]["name"]}')

    # break

print(f'{len(spotify_tracks)}/{len(tracks)}')
[f'{spt["name"]}: {",".join([artist["name"] for artist in spt["artists"]])}' for spt in spotify_tracks[:2]]

In [ ]:
[
    (st["id"], st["uri"], st["name"], 
    [a["name"] for a in st["artists"]])
    for st in spotify_tracks
]